In [ ]:
from matplotlib import *
from __future__ import division
%matplotlib inline
import pandas as pd
import numpy as np
import csv
from matplotlib import pyplot as plt
import pyproj
from sklearn.metrics.pairwise import pairwise_distances



## create fake populations and distances

In [ ]:

# Sample Data: Population of locations (Replace with WorldPop data)
locations = ['A', 'B', 'C', 'D', 'E']
population = [10000, 5000, 20000, 15000, 12000]
distances = {
    ('A', 'B'): 10, ('A', 'C'): 30, ('A', 'D'): 50, ('A', 'E'): 70,
    ('B', 'C'): 20, ('B', 'D'): 40, ('B', 'E'): 60,
    ('C', 'D'): 15, ('C', 'E'): 35,
    ('D', 'E'): 25
}

# Convert to DataFrame
pop_df = pd.DataFrame({'Location': locations, 'Population': population})
dist_df = pd.DataFrame([(k[0], k[1], v) for k, v in distances.items()], columns=['From', 'To', 'Distance'])



## Build your Gravity Model 

The number of trips occurring from i to j are inversely proportional to the distance that divides i and j and where masses of origin and destination are the respective populations.


The form with the power law deterrence functions is written as:  
$\Large T_{ij} = K \frac{m_i^\alpha m_j^\beta}{d^\gamma}$

The form with the exponential deterrence functions is written as:  
$\Large T_{ij} = K m_i^\alpha m_j^\beta e^{-d/d_0}$


where $\alpha$ and $\beta$ modulate the attractiveness of masses  
$\gamma$ regulates the decay of the gravity force with distance  
and $d_0$ represents the typical distance travelled by individuals

### create a table containing the two gravity models predicted flows between i and j

In [ ]:

def gravity_model(pop_df, dist_df, alpha=1, beta=1):
    flows = []
    for _, row in dist_df.iterrows():
        
    dist_df['Gravity Flow PL'] = flows
    return dist_df

def gravity_model_exp(pop_df, dist_df, alpha=1, beta=1):
    flows = []
    for _, row in dist_df.iterrows():
        
    dist_df['Gravity Flow Exp'] = flows
    return dist_df

alpha=1
beta=1
d0=5
gravity_results = gravity_model(pop_df, dist_df)
gravity_results = gravity_model_exp(pop_df, dist_df)

print("Gravity Model Results:")
print(gravity_results)


## Build your Radiation Model

The number of trips occurring from i to j is controlled by the formula


$\Large T_{ij} = T_i \frac{(m_i m_j)}{(m_i + s_{ij})(m_i + m_j + s_{ij})}$




where $m_i$ and $m_j$ are the populations of i and j and $s_{ij}$ is the intervening population


### Add the radiation model predicted flows to the table of results   
assume Ti = population_i

In [ ]:

def radiation_model(pop_df, dist_df):
    flows = []
    for _, row in dist_df.iterrows():
        
    dist_df['Radiation Flow'] = flows
    return dist_df

radiation_results = radiation_model(pop_df, dist_df)
print("\nRadiation Model Results:")
print(radiation_results)


## Now generate flows from population distribution from WorldPop in Turin

### Get data from WorldPop https://hub.worldpop.org/geodata/summary?id=36916

In [ ]:
worldpop_df0 = pd.read_csv('ppp_ITA_2020_1km_Aggregated_UNadj.csv')
print(len(worldpop_df0))

plt.scatter(worldpop_df0.X,worldpop_df0.Y,c=np.log(worldpop_df0.Z+1), cmap=cm.viridis, s=.02, marker='s')
plt.gca().set_aspect('equal')

plt.axis('off')

## Let's focus on Turin

In [ ]:
x0 = 7.61000
x1 = 7.7100
y0 = 45.0200
y1 = 45.12322

worldpop_df = worldpop_df0[(worldpop_df0.X<x1)&(worldpop_df0.X>x0)&(worldpop_df0.Y<y1)&(worldpop_df0.Y>y0)]
print(len(worldpop_df))

#project to mercator
P = pyproj.Proj(proj='utm', zone=31, ellps='WGS84', preserve_units=True)
worldpop_df['x'] = P(worldpop_df.X,worldpop_df.Y)[0]-P(x0,45)[0]
worldpop_df['y'] = P(worldpop_df.X,worldpop_df.Y)[1]-P(6,y0)[1]


# Sample Data: Population of locations (Replace with WorldPop data)
locations = worldpop_df.index.tolist()
population = worldpop_df['Z'].tolist()

distances = pairwise_distances(worldpop_df[['x','y']], metric='euclidean')

plt.scatter(worldpop_df.X,worldpop_df.Y,c=worldpop_df.Z, cmap=cm.viridis, s=395, marker='s')
plt.gca().set_aspect('equal')

In [ ]:
# Convert to DataFrame
pop_df = pd.DataFrame({'Location': locations, 'Population': population})
pop_dict = pop_df.set_index('Location')['Population'].to_dict()
dist_df = pd.DataFrame(distances, index=locations, columns=locations)

In [ ]:
distance_df = dist_df.unstack().reset_index()
distance_df = distance_df.rename(columns={'level_0':'origin','level_1':'destination',0:'distance'})

In [ ]:
distance_df['pop orig'] = distance_df['origin'].apply(lambda x: pop_dict[x])
distance_df['pop dest'] = distance_df['destination'].apply(lambda x: pop_dict[x])

distance_df = distance_df[(distance_df['pop orig']>0) & (distance_df['pop dest']>0)]

In [ ]:
distance_df.head()

### Use your gravity model to generate flows with the WorldPop data  
use the distance_df

In [ ]:

# Gravity Model Function
alpha=1
beta=1
gamma=2
d0=200 #meters
def Gravity_pow(x, y, d, alpha, beta, gamma):
    return 

def Gravity_exp(x, y, d, alpha, beta, d0):
    return 

distance_df['gravity model pow'] = distance_df[...
distance_df['gravity model exp'] = distance_df[...
                                               
distance_df.replace([np.inf, -np.inf], np.nan, inplace=True)

### Use your radiation model to generate flows with the WorldPop data

In [ ]:
def get_s(origin,destination):
    df_orig = distance_df[(distance_df.origin==origin)]
    distance = distance_df[(distance_df.origin==origin)&(distance_df.destination==destination)].distance.values[0]
    loc_s = set(df_orig[df_orig['distance']<distance]['destination'].values)
    pop_s = pop_df[pop_df.Location.isin(loc_s)]['Population'].sum()
    return pop_s
        
    
pops_s = []
for i in distance_df.origin.unique():
    print(i)
    for j in distance_df.destination.unique():
        if j>=i:
            pops_s.append([i,j,get_s(i,j)])
            pops_s.append([j,i,get_s(i,j)])

#### merge the intervening population s into the distance dataframe

In [ ]:
pop_s = pd.DataFrame(pops_s, columns=['origin','destination','s'])

In [ ]:
new_df = pd.merge(
    left=distance_df, 
    right=pop_s,
    how='left',
    left_on=['origin', 'destination'],
    right_on=['origin', 'destination']
)

In [ ]:

# Radiation Model Function
def radiation_model(pop_i, pop_j, pop_s):
    return ...
        

new_df['Radiation model'] = new_df[...


In [ ]:
new_df.head()

### Check if scikit-mobility fits the parameters you used in the Gravity models

In [ ]:
import skmob
from skmob.utils import utils, constants
import geopandas as gpd
from skmob.models import gravity

#### scikit data structure wants a geopandas geometry to compute distances in lat, lon

In [ ]:
gdf = gpd.GeoDataFrame(worldpop_df, geometry=gpd.points_from_xy(worldpop_df['X'], worldpop_df['Y']))
gdf = gdf.reset_index().rename(columns={'Z':'population','index':'tile_ID'})
gdf

### define dataframe with only columns: origin, destination, flow 

In [ ]:
flow_df = new_df[['origin','destination','gravity model exp']].rename(columns={'gravity model exp':'flow'})

# load data about mobility flows into a FlowDataFrame
fdf = skmob.FlowDataFrame(flow_df,tessellation=gdf, tile_id='tile_ID')


### Check if scikit-mobility finds the correct parameters you used to generate the flows with the Gravity models  
use the examples in the notebook mobility_models in the repository

In [ ]:
# fit the parameters of the Gravity model from real fluxes
gravity_fit_exp = gravity.Gravity(...
print(gravity_fit_exp)

In [ ]:
fdf.head()

In [ ]:
# fit the parameters of the Gravity model from real fluxes
gravity_fit_exp.fit(fdf, relevance_column='population')
print(gravity_fit_exp)


the parameter "deterrence function" is the exponent d0 of the exponential function, origin and destination exp are the alpha and beta parameters above